# Livestock Health Records: Tracking Your Flock

This notebook helps you build and maintain organized health records for your **Katahdin hair sheep** flock. Good records are not just paperwork -- they are a management tool that helps you make better breeding, culling, and treatment decisions.

**Why does this matter for Katahdin producers?**

- Katahdins are prized for parasite resistance, but you need records to prove which animals are actually resistant and which are not
- FAMACHA scoring over time reveals which ewes consistently handle parasites on their own
- Weight and growth data identifies your best genetics for breeding stock selection
- Treatment records track withdrawal periods so you know when animals are clear for market
- Organized records are required for many direct-market certifications and veterinary consultations

Whether you have 10 sheep or 100, keeping consistent records turns your flock from a guessing game into a data-driven operation.

## How to Run This Notebook

1. Make sure you have **uv** installed (see [uv docs](https://docs.astral.sh/uv/))
2. From the project root, install dependencies with the data extras:
   ```bash
   uv sync --extra data
   ```
3. Start Jupyter:
   ```bash
   uv run jupyter notebook
   ```
4. Open this notebook and run cells in order from top to bottom
5. Replace the sample animal names, IDs, and data with your own flock records

Each section explains what the code does in plain language before you run it.

## Setup

Load the tools this notebook needs. Run this cell once at the start.

In [ ]:
# Standard library
from datetime import datetime

# Data handling
import pandas as pd

try:
    import matplotlib.pyplot as plt
    import numpy as np

    HAS_MATPLOTLIB = True
    plt.rcParams["figure.figsize"] = (10, 6)
    plt.rcParams["figure.dpi"] = 100
except ImportError:
    HAS_MATPLOTLIB = False
    print("Optional: pip install matplotlib for charts")

try:
    from tabulate import tabulate

    HAS_TABULATE = True
except ImportError:
    HAS_TABULATE = False
    print("Optional: pip install tabulate for prettier tables")

# Configuration
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 30)

print(f"Notebook ready - {datetime.now().strftime('%B %d, %Y')}")

## Recording Health Events

The foundation of good livestock records is capturing three types of health events:

1. **Vaccinations** -- what was given, when, to whom
2. **Treatments** -- illnesses, medications, withdrawal periods
3. **Vet visits** -- professional consultations and recommendations

Below we set up a sample flock of 10 Katahdin sheep and record health events for them. **Replace this sample data with your actual flock.**

### The Flock Roster

First, let's define our animals. Each animal gets a unique ID, a name (for those of us who name our sheep), and basic information.

In [ ]:
# Sample Katahdin flock roster
# Replace with your actual animals
raw_flock = pd.DataFrame([
    {"animal_id": "KAT-001", "name": "Clover",   "sex": "Ewe",  "birth_date": "2022-03-10", "sire": "KAT-R01", "dam": "KAT-050", "color": "White"},
    {"animal_id": "KAT-002", "name": "Maple",    "sex": "Ewe",  "birth_date": "2022-03-14", "sire": "KAT-R01", "dam": "KAT-051", "color": "Brown"},
    {"animal_id": "KAT-003", "name": "Hazel",    "sex": "Ewe",  "birth_date": "2023-02-28", "sire": "KAT-R02", "dam": "KAT-001", "color": "White"},
    {"animal_id": "KAT-004", "name": "Fern",     "sex": "Ewe",  "birth_date": "2023-03-05", "sire": "KAT-R02", "dam": "KAT-002", "color": "Brown"},
    {"animal_id": "KAT-005", "name": "Thistle",  "sex": "Ewe",  "birth_date": "2024-03-01", "sire": "KAT-R02", "dam": "KAT-001", "color": "White/Brown"},
    {"animal_id": "KAT-006", "name": "Rosemary", "sex": "Ewe",  "birth_date": "2024-03-08", "sire": "KAT-R03", "dam": "KAT-002", "color": "White"},
    {"animal_id": "KAT-R02", "name": "Flint",    "sex": "Ram",  "birth_date": "2021-02-20", "sire": "External", "dam": "External", "color": "Brown"},
    {"animal_id": "KAT-R03", "name": "Cedar",    "sex": "Ram",  "birth_date": "2023-03-12", "sire": "KAT-R01", "dam": "KAT-050", "color": "White"},
    {"animal_id": "KAT-L01", "name": "Birch",    "sex": "Ram Lamb", "birth_date": "2025-03-02", "sire": "KAT-R02", "dam": "KAT-003", "color": "White"},
    {"animal_id": "KAT-L02", "name": "Sage",     "sex": "Ewe Lamb", "birth_date": "2025-03-05", "sire": "KAT-R03", "dam": "KAT-004", "color": "Brown"},
])

print(f"Flock roster: {len(raw_flock)} animals")
if HAS_TABULATE:
    print(tabulate(
        raw_flock[["animal_id", "name", "sex", "birth_date", "color"]],
        headers=["ID", "Name", "Sex", "Birth Date", "Color"],
        tablefmt="simple", showindex=False,
    ))
else:
    print(raw_flock[["animal_id", "name", "sex", "birth_date", "color"]].to_string())

### Vaccination Records

Vaccinations are the backbone of a preventive health program. For Katahdin flocks, the most common vaccines are:

- **CDT** (Clostridium perfringens types C & D + Tetanus) -- the core vaccine every sheep should get
- **CLA** (Caseous Lymphadenitis) -- for flocks with a history of abscesses

Below we record vaccination events. Note the route (SubQ = subcutaneous, IM = intramuscular) and dose.

In [ ]:
# Vaccination records
raw_vaccinations = pd.DataFrame([
    {"animal_id": "KAT-001", "date": "2025-03-01", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 2.0, "administered_by": "Maria"},
    {"animal_id": "KAT-002", "date": "2025-03-01", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 2.0, "administered_by": "Maria"},
    {"animal_id": "KAT-003", "date": "2025-03-01", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 2.0, "administered_by": "Maria"},
    {"animal_id": "KAT-004", "date": "2025-03-01", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 2.0, "administered_by": "Maria"},
    {"animal_id": "KAT-005", "date": "2025-03-01", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 2.0, "administered_by": "Maria"},
    {"animal_id": "KAT-006", "date": "2025-03-01", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 2.0, "administered_by": "Maria"},
    {"animal_id": "KAT-R02", "date": "2025-03-01", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 2.0, "administered_by": "Maria"},
    {"animal_id": "KAT-R03", "date": "2025-03-01", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 2.0, "administered_by": "Maria"},
    {"animal_id": "KAT-001", "date": "2025-04-15", "vaccine": "CLA",  "route": "SubQ", "dose_ml": 1.0, "administered_by": "Maria"},
    {"animal_id": "KAT-002", "date": "2025-04-15", "vaccine": "CLA",  "route": "SubQ", "dose_ml": 1.0, "administered_by": "Maria"},
    {"animal_id": "KAT-L01", "date": "2025-04-02", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 1.0, "administered_by": "Maria"},
    {"animal_id": "KAT-L02", "date": "2025-04-05", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 1.0, "administered_by": "Maria"},
    {"animal_id": "KAT-L01", "date": "2025-05-02", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 2.0, "administered_by": "Maria"},
    {"animal_id": "KAT-L02", "date": "2025-05-05", "vaccine": "CDT",  "route": "SubQ", "dose_ml": 2.0, "administered_by": "Maria"},
])

print(f"Total vaccination records: {len(raw_vaccinations)}")
print()
vaccinations_by_vaccine = raw_vaccinations.groupby("vaccine").size()
print("Vaccinations by type:")
for vaccine, count in vaccinations_by_vaccine.items():
    print(f"  {vaccine}: {count} doses administered")

### Treatment Records

When an animal gets sick or needs medication, we record the condition, what was given, and -- critically -- the **withdrawal period** in days. This is the number of days after treatment before the animal's meat or milk is safe for market.

Common conditions in Katahdin flocks include parasites (despite their resistance, some animals still need help), foot rot, and occasional respiratory issues.

In [ ]:
# Treatment records
raw_treatments = pd.DataFrame([
    {"animal_id": "KAT-005", "date": "2025-06-15", "condition": "Barber pole worm (Haemonchus)",
     "treatment": "Cydectin (Moxidectin) oral drench", "withdrawal_days": 14, "cost": 3.50},
    {"animal_id": "KAT-006", "date": "2025-06-15", "condition": "Barber pole worm (Haemonchus)",
     "treatment": "Cydectin (Moxidectin) oral drench", "withdrawal_days": 14, "cost": 3.50},
    {"animal_id": "KAT-R02", "date": "2025-07-20", "condition": "Foot rot",
     "treatment": "Hoof trim + zinc sulfate foot bath", "withdrawal_days": 0, "cost": 5.00},
    {"animal_id": "KAT-003", "date": "2025-08-10", "condition": "Nasal discharge / respiratory",
     "treatment": "Draxxin (Tulathromycin) injection", "withdrawal_days": 18, "cost": 12.00},
    {"animal_id": "KAT-005", "date": "2025-09-01", "condition": "Barber pole worm (Haemonchus)",
     "treatment": "Prohibit (Levamisole) oral drench", "withdrawal_days": 3, "cost": 2.00},
    {"animal_id": "KAT-004", "date": "2025-10-05", "condition": "Bottle jaw / anemia",
     "treatment": "Valbazen (Albendazole) + iron supplement", "withdrawal_days": 9, "cost": 6.50},
    {"animal_id": "KAT-R02", "date": "2025-11-15", "condition": "Foot scald",
     "treatment": "Hoof trim + Koppertox topical", "withdrawal_days": 0, "cost": 4.00},
])

# Calculate withdrawal clear dates
raw_treatments["date_parsed"] = pd.to_datetime(raw_treatments["date"])
raw_treatments["clear_date"] = (
    raw_treatments["date_parsed"] +
    pd.to_timedelta(raw_treatments["withdrawal_days"], unit="D")
).dt.strftime("%Y-%m-%d")

print("Treatment Records")
print("=" * 60)
if HAS_TABULATE:
    print(tabulate(
        raw_treatments[["animal_id", "date", "condition", "treatment", "withdrawal_days", "clear_date"]],
        headers=["Animal", "Date", "Condition", "Treatment", "W/D Days", "Clear Date"],
        tablefmt="simple", showindex=False,
        maxcolwidths=[10, 12, 25, 30, 8, 12],
    ))
else:
    print(raw_treatments[["animal_id", "date", "condition", "treatment", "withdrawal_days", "clear_date"]].to_string())

### Vet Visit Records

Professional veterinary visits should be documented even when everything is routine. This creates a paper trail and captures recommendations you can refer back to.

In [ ]:
# Vet visit records
raw_vet_visits = pd.DataFrame([
    {"date": "2025-03-01", "reason": "Annual wellness check and vaccination",
     "findings": "Flock in good condition. BCS 3-3.5 across all ewes.",
     "recommendations": "Continue CDT annually. Add CLA for older ewes.",
     "cost": 150.00},
    {"date": "2025-06-15", "reason": "FAMACHA training and parasite evaluation",
     "findings": "KAT-005 and KAT-006 at FAMACHA 4, mild anemia.",
     "recommendations": "Drench with Moxidectin. Re-check in 14 days. Consider culling if recurring.",
     "cost": 125.00},
    {"date": "2025-08-10", "reason": "Respiratory issue in KAT-003",
     "findings": "Mild pneumonia, temp 104.5F. No spread to others.",
     "recommendations": "Draxxin injection. Isolate 5 days. Monitor flock temps.",
     "cost": 175.00},
    {"date": "2025-11-15", "reason": "Pre-breeding soundness exam for rams",
     "findings": "Flint (R02) - persistent foot issues. Cedar (R03) - sound, good condition.",
     "recommendations": "Limit Flint's use this season. Cedar as primary sire.",
     "cost": 200.00},
])

total_vet_cost = raw_vet_visits["cost"].sum()
print(f"Vet visits this year: {len(raw_vet_visits)}")
print(f"Total vet costs: ${total_vet_cost:.2f}")
print()
if HAS_TABULATE:
    print(tabulate(
        raw_vet_visits[["date", "reason", "cost"]],
        headers=["Date", "Reason", "Cost"],
        tablefmt="simple", showindex=False,
    ))
else:
    print(raw_vet_visits[["date", "reason", "cost"]].to_string())

## Weight Tracking and Growth Curves

Weighing lambs at regular intervals lets you calculate **Average Daily Gain (ADG)**, which is one of the best measures of both lamb genetics and ewe mothering ability. Katahdins are known for fast growth on grass -- tracking weights tells you if your flock is meeting that potential.

Standard weigh points are birth, 30 days, 60 days, 90 days, and weaning (typically around 90-120 days for hair sheep).

In [ ]:
# Weight records for 2025 lamb crop (pounds)
raw_weights = pd.DataFrame([
    {"animal_id": "KAT-L01", "name": "Birch",  "sex": "Ram Lamb",
     "birth_wt": 8.5, "day30_wt": 28, "day60_wt": 48, "day90_wt": 68, "weaning_wt": 72},
    {"animal_id": "KAT-L02", "name": "Sage",   "sex": "Ewe Lamb",
     "birth_wt": 7.0, "day30_wt": 24, "day60_wt": 42, "day90_wt": 58, "weaning_wt": 62},
    {"animal_id": "KAT-L03", "name": "Willow", "sex": "Ewe Lamb",
     "birth_wt": 7.5, "day30_wt": 26, "day60_wt": 44, "day90_wt": 60, "weaning_wt": 65},
    {"animal_id": "KAT-L04", "name": "Oak",    "sex": "Ram Lamb",
     "birth_wt": 9.0, "day30_wt": 30, "day60_wt": 52, "day90_wt": 72, "weaning_wt": 78},
    {"animal_id": "KAT-L05", "name": "Ivy",    "sex": "Ewe Lamb",
     "birth_wt": 6.5, "day30_wt": 22, "day60_wt": 38, "day90_wt": 52, "weaning_wt": 56},
])

# Calculate Average Daily Gain (birth to weaning, ~100 days)
weaning_age_days = 100
raw_weights["adg_lbs"] = (
    (raw_weights["weaning_wt"] - raw_weights["birth_wt"]) / weaning_age_days
).round(2)

print("Lamb Weight Records - 2025 Crop")
print("=" * 55)
if HAS_TABULATE:
    print(tabulate(
        raw_weights[["animal_id", "name", "sex", "birth_wt", "weaning_wt", "adg_lbs"]],
        headers=["ID", "Name", "Sex", "Birth (lbs)", "Weaning (lbs)", "ADG (lbs/day)"],
        tablefmt="simple", showindex=False,
    ))
else:
    print(raw_weights[["animal_id", "name", "sex", "birth_wt", "weaning_wt", "adg_lbs"]].to_string())
print(f"\nFlock average ADG: {raw_weights['adg_lbs'].mean():.2f} lbs/day")

### Growth Curves

This chart shows each lamb's weight at each weigh point. Lambs that fall behind the group may need attention -- check their dam's milk production and overall health.

In [ ]:
# Growth curve chart
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(10, 6))

    weigh_points = ["Birth", "30 days", "60 days", "90 days", "Weaning"]
    weight_cols = ["birth_wt", "day30_wt", "day60_wt", "day90_wt", "weaning_wt"]
    ram_colors = ["#2980b9", "#1a5276"]
    ewe_colors = ["#e74c3c", "#c0392b", "#a93226"]
    ram_idx, ewe_idx = 0, 0

    for _, row in raw_weights.iterrows():
        weights = [row[col] for col in weight_cols]
        if "Ram" in row["sex"]:
            color = ram_colors[ram_idx % len(ram_colors)]
            ram_idx += 1
        else:
            color = ewe_colors[ewe_idx % len(ewe_colors)]
            ewe_idx += 1
        ax.plot(weigh_points, weights, marker="o", linewidth=2,
                label=f"{row['name']} ({row['sex']})", color=color)

    ax.set_title("2025 Lamb Growth Curves", fontsize=14)
    ax.set_ylabel("Weight (lbs)")
    ax.set_xlabel("Weigh Point")
    ax.legend(loc="upper left")
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart")

## Body Condition Scoring

**Body Condition Score (BCS)** is a hands-on assessment of how much fat and muscle an animal is carrying. For sheep, the scale runs from 1 to 5:

| BCS | Condition | What You Feel Along the Spine |
|-----|-----------|-------------------------------|
| 1 | Emaciated | Sharp spine, no fat, fingers go under easily |
| 2 | Thin | Spine prominent, slight fat cover |
| 3 | Ideal | Spine felt with firm pressure, smooth fat cover |
| 4 | Fat | Spine hard to feel, thick fat layer |
| 5 | Obese | Spine cannot be felt, excessive fat |

For Katahdin ewes:
- **Breeding**: Aim for BCS 3.0-3.5 ("flushing" condition)
- **Late gestation**: Should not drop below 2.5
- **Lactation**: Some loss is normal, but below 2.0 is concerning

In [ ]:
# Body Condition Score tracking
raw_bcs = pd.DataFrame([
    {"animal_id": "KAT-001", "name": "Clover",   "2025-03": 3.0, "2025-06": 3.0, "2025-09": 3.5, "2025-12": 3.0, "2026-02": 3.0},
    {"animal_id": "KAT-002", "name": "Maple",    "2025-03": 3.5, "2025-06": 3.0, "2025-09": 3.0, "2025-12": 3.5, "2026-02": 3.5},
    {"animal_id": "KAT-003", "name": "Hazel",    "2025-03": 3.0, "2025-06": 2.5, "2025-09": 2.5, "2025-12": 3.0, "2026-02": 3.0},
    {"animal_id": "KAT-004", "name": "Fern",     "2025-03": 3.0, "2025-06": 2.5, "2025-09": 2.0, "2025-12": 2.5, "2026-02": 2.5},
    {"animal_id": "KAT-005", "name": "Thistle",  "2025-03": 3.0, "2025-06": 2.5, "2025-09": 2.5, "2025-12": 3.0, "2026-02": 3.0},
    {"animal_id": "KAT-006", "name": "Rosemary", "2025-03": 3.5, "2025-06": 3.0, "2025-09": 3.0, "2025-12": 3.0, "2026-02": 3.0},
])

print("Body Condition Scores - Ewes")
print("=" * 60)
if HAS_TABULATE:
    print(tabulate(
        raw_bcs,
        headers=["ID", "Name", "Mar 25", "Jun 25", "Sep 25", "Dec 25", "Feb 26"],
        tablefmt="simple", showindex=False,
    ))
else:
    print(raw_bcs.to_string())

### BCS Trend Chart

This chart shows how each ewe's body condition has changed over time. Watch for animals dropping below 2.5 -- they may need supplemental feeding or a health check.

In [ ]:
# BCS trend chart
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(10, 6))

    bcs_dates = ["Mar 25", "Jun 25", "Sep 25", "Dec 25", "Feb 26"]
    bcs_cols = ["2025-03", "2025-06", "2025-09", "2025-12", "2026-02"]
    colors = ["#2ecc71", "#3498db", "#9b59b6", "#e67e22", "#e74c3c", "#1abc9c"]

    for i, (_, row) in enumerate(raw_bcs.iterrows()):
        scores = [row[col] for col in bcs_cols]
        ax.plot(bcs_dates, scores, marker="o", linewidth=2,
                label=row["name"], color=colors[i])

    ax.axhline(y=3.0, color="green", linestyle="--", alpha=0.5, label="Ideal (3.0)")
    ax.axhline(y=2.5, color="orange", linestyle="--", alpha=0.5, label="Watch (2.5)")
    ax.axhline(y=2.0, color="red", linestyle="--", alpha=0.5, label="Concern (2.0)")

    ax.set_title("Ewe Body Condition Score Trends", fontsize=14)
    ax.set_ylabel("BCS (1-5)")
    ax.set_xlabel("Assessment Period")
    ax.set_ylim(1.5, 4.0)
    ax.legend(loc="upper right", fontsize=8, ncol=3)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart")

## FAMACHA Assessment Log

**FAMACHA** is a hands-on method for assessing parasite load in sheep by examining the color of the lower eyelid mucous membranes. It was developed specifically for detecting *Haemonchus contortus* (barber pole worm), the most dangerous internal parasite for sheep.

The FAMACHA scale:

| Score | Eyelid Color | Meaning | Action |
|-------|-------------|---------|--------|
| 1 | Red | Healthy, no anemia | No treatment needed |
| 2 | Red-Pink | Acceptable | Monitor, no treatment |
| 3 | Pink | Borderline anemia | Monitor closely, consider treatment |
| 4 | Pink-White | Anemic | Deworm immediately |
| 5 | White | Severely anemic | Deworm immediately, may need supportive care |

For **Katahdin sheep**, one of their key breed traits is parasite resistance. FAMACHA scoring over multiple seasons reveals which animals truly carry that trait. Ewes that consistently score 1-2 without deworming are your best genetics. Animals that repeatedly score 4-5 should be culled from the breeding flock.

In [ ]:
# FAMACHA assessment log across the grazing season
raw_famacha = pd.DataFrame([
    {"animal_id": "KAT-001", "name": "Clover",   "may": 1, "jun": 1, "jul": 2, "aug": 1, "sep": 1, "oct": 1},
    {"animal_id": "KAT-002", "name": "Maple",    "may": 1, "jun": 2, "jul": 2, "aug": 2, "sep": 1, "oct": 1},
    {"animal_id": "KAT-003", "name": "Hazel",    "may": 2, "jun": 2, "jul": 3, "aug": 2, "sep": 2, "oct": 2},
    {"animal_id": "KAT-004", "name": "Fern",     "may": 2, "jun": 3, "jul": 3, "aug": 4, "sep": 3, "oct": 2},
    {"animal_id": "KAT-005", "name": "Thistle",  "may": 2, "jun": 3, "jul": 4, "aug": 3, "sep": 4, "oct": 2},
    {"animal_id": "KAT-006", "name": "Rosemary", "may": 1, "jun": 2, "jul": 4, "aug": 2, "sep": 2, "oct": 1},
    {"animal_id": "KAT-R02", "name": "Flint",    "may": 1, "jun": 1, "jul": 2, "aug": 2, "sep": 1, "oct": 1},
    {"animal_id": "KAT-R03", "name": "Cedar",    "may": 1, "jun": 1, "jul": 1, "aug": 2, "sep": 1, "oct": 1},
])

# Calculate season average
score_cols = ["may", "jun", "jul", "aug", "sep", "oct"]
raw_famacha["season_avg"] = raw_famacha[score_cols].mean(axis=1).round(1)
raw_famacha["times_treated"] = (raw_famacha[score_cols] >= 4).sum(axis=1)

print("FAMACHA Assessment Log - 2025 Grazing Season")
print("=" * 70)
if HAS_TABULATE:
    print(tabulate(
        raw_famacha[["animal_id", "name", "may", "jun", "jul", "aug", "sep", "oct", "season_avg", "times_treated"]],
        headers=["ID", "Name", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Avg", "Treated"],
        tablefmt="simple", showindex=False,
    ))
else:
    print(raw_famacha[["animal_id", "name", "may", "jun", "jul", "aug", "sep", "oct", "season_avg", "times_treated"]].to_string())

### FAMACHA Decision Guide

Based on the season's data, here is a plain-language assessment of each animal's parasite resilience. This is exactly the kind of information you need for breeding and culling decisions.

In [ ]:
# Generate plain-language FAMACHA assessments
print("FAMACHA Season Summary - Breeding Recommendations")
print("=" * 60)

for _, row in raw_famacha.iterrows():
    avg = row["season_avg"]
    treated = row["times_treated"]

    if avg <= 1.5 and treated == 0:
        status = "EXCELLENT - strong parasite resistance, top breeding stock"
    elif avg <= 2.0 and treated == 0:
        status = "GOOD - solid resistance, keep in breeding program"
    elif avg <= 2.5 and treated <= 1:
        status = "FAIR - adequate, monitor next season"
    elif treated >= 2:
        status = "POOR - recurring issues, consider culling from breeding flock"
    else:
        status = "WATCH - needed treatment, evaluate again next year"

    print(f"  {row['name']} ({row['animal_id']}): avg {avg}, treated {treated}x")
    print(f"    -> {status}")
    print()

### FAMACHA Trend Over the Season

This chart tracks each animal's FAMACHA score month by month. Scores climbing toward 4-5 during summer indicate the animal is losing the battle with parasites.

In [ ]:
# FAMACHA trend chart
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(10, 6))

    months = ["May", "Jun", "Jul", "Aug", "Sep", "Oct"]
    colors = ["#27ae60", "#2980b9", "#8e44ad", "#e67e22",
              "#e74c3c", "#1abc9c", "#34495e", "#f39c12"]

    for i, (_, row) in enumerate(raw_famacha.iterrows()):
        scores = [row[col] for col in score_cols]
        ax.plot(months, scores, marker="o", linewidth=2,
                label=row["name"], color=colors[i])

    # Treatment threshold lines
    ax.axhline(y=4, color="red", linestyle="--", alpha=0.5, label="Treat (4+)")
    ax.axhline(y=3, color="orange", linestyle="--", alpha=0.3, label="Watch (3)")

    ax.set_title("FAMACHA Scores - 2025 Grazing Season", fontsize=14)
    ax.set_ylabel("FAMACHA Score (1=Healthy, 5=Severe)")
    ax.set_xlabel("Month")
    ax.set_ylim(0.5, 5.5)
    ax.invert_yaxis()
    ax.legend(loc="lower left", fontsize=8, ncol=3)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart")

## Seasonal Health Pattern Identification

By looking at when health events happen across the year, you can anticipate problems and plan ahead. Most sheep operations see predictable seasonal patterns -- parasites peak in warm/wet months, foot issues in muddy seasons, and respiratory problems in cold weather.

In [ ]:
# Aggregate treatment events by month and condition
raw_treatments["month"] = pd.to_datetime(raw_treatments["date"]).dt.month
raw_treatments["month_name"] = pd.to_datetime(raw_treatments["date"]).dt.strftime("%b")

# Categorize conditions into broad types


def categorize_condition(condition):
    condition_lower = condition.lower()
    if "worm" in condition_lower or "parasite" in condition_lower or "anemia" in condition_lower or "bottle jaw" in condition_lower:
        return "Parasites"
    elif "foot" in condition_lower or "hoof" in condition_lower:
        return "Foot/Hoof"
    elif "respiratory" in condition_lower or "pneumonia" in condition_lower or "nasal" in condition_lower:
        return "Respiratory"
    else:
        return "Other"


raw_treatments["category"] = raw_treatments["condition"].apply(categorize_condition)

events_by_month = raw_treatments.groupby(["month", "month_name", "category"]).size().reset_index(name="count")

print("Health Events by Month and Category")
print("=" * 40)
if HAS_TABULATE:
    print(tabulate(
        events_by_month[["month_name", "category", "count"]],
        headers=["Month", "Category", "Events"],
        tablefmt="simple", showindex=False,
    ))
else:
    print(events_by_month[["month_name", "category", "count"]].to_string())

### Seasonal Health Events Chart

This chart shows when different types of health issues occur across the year. Use it to plan your preventive care calendar.

In [ ]:
# Bar chart of health events by month and category
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(10, 6))

    all_months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                  "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    categories = ["Parasites", "Foot/Hoof", "Respiratory"]
    cat_colors = {"Parasites": "#e74c3c", "Foot/Hoof": "#3498db", "Respiratory": "#f39c12"}

    x = np.arange(len(all_months))
    bar_width = 0.25

    for i, cat in enumerate(categories):
        cat_data = events_by_month[events_by_month["category"] == cat]
        counts = []
        for m_idx in range(1, 13):
            match = cat_data[cat_data["month"] == m_idx]
            counts.append(match["count"].sum() if len(match) > 0 else 0)
        ax.bar(x + i * bar_width, counts, bar_width,
               label=cat, color=cat_colors[cat], alpha=0.8)

    ax.set_title("Health Events by Month and Type", fontsize=14)
    ax.set_ylabel("Number of Events")
    ax.set_xlabel("Month")
    ax.set_xticks(x + bar_width)
    ax.set_xticklabels(all_months)
    ax.legend()
    ax.grid(True, axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart")

print("\nHigh-risk periods identified:")
print("  - Parasites: June through September (warm, humid months)")
print("  - Foot/Hoof: July and November (wet/muddy conditions)")
print("  - Respiratory: August (heat stress or dust)")

## Generating Summary Reports

When you visit the vet or need to review your flock's health history, a formatted summary report is invaluable. This section pulls together the key information into a report you can print or share.

In [ ]:
# Generate a vet consultation summary report
print("=" * 70)
print("       FLOCK HEALTH SUMMARY REPORT")
print(f"       Prepared: {datetime.now().strftime('%B %d, %Y')}")
print("=" * 70)

print(f"\nFlock size: {len(raw_flock)} animals")
print("Breeds: Katahdin Hair Sheep")
print(f"Ewes: {len(raw_flock[raw_flock['sex'].str.contains('Ewe')])}")
print(f"Rams: {len(raw_flock[raw_flock['sex'].str.contains('Ram')])}")

### Treatment History Summary

A breakdown of all treatments given during the record period, with costs.

In [ ]:
# Treatment summary
print("\nTREATMENT HISTORY")
print("-" * 70)
if HAS_TABULATE:
    print(tabulate(
        raw_treatments[["date", "animal_id", "condition", "treatment", "cost"]],
        headers=["Date", "Animal", "Condition", "Treatment", "Cost"],
        tablefmt="grid", showindex=False,
        maxcolwidths=[12, 10, 25, 30, 8],
    ))
else:
    print(raw_treatments[["date", "animal_id", "condition", "treatment", "cost"]].to_string())

total_treatment_cost = raw_treatments["cost"].sum()
cost_by_category = raw_treatments.groupby("category")["cost"].sum()

print(f"\nTotal treatment costs: ${total_treatment_cost:.2f}")
print("Cost breakdown:")
for cat, cost in cost_by_category.items():
    print(f"  {cat}: ${cost:.2f}")

### Animals Requiring Attention

This flags animals that may need follow-up based on treatment frequency, FAMACHA scores, or declining body condition.

In [ ]:
# Flag animals needing attention
print("\nANIMALS REQUIRING ATTENTION")
print("-" * 50)

# Animals treated multiple times
treatment_counts = raw_treatments.groupby("animal_id").size()
repeat_treated = treatment_counts[treatment_counts > 1]

if len(repeat_treated) > 0:
    print("\nRepeat treatments (multiple health events):")
    for animal_id, count in repeat_treated.items():
        name = raw_flock.loc[raw_flock["animal_id"] == animal_id, "name"]
        name_str = name.iloc[0] if len(name) > 0 else "Unknown"
        print(f"  {name_str} ({animal_id}): {count} treatments")

# FAMACHA concerns
famacha_concerns = raw_famacha[raw_famacha["season_avg"] >= 2.5]
if len(famacha_concerns) > 0:
    print("\nFAMACHA season average 2.5 or higher:")
    for _, row in famacha_concerns.iterrows():
        print(f"  {row['name']} ({row['animal_id']}): avg {row['season_avg']}")

# BCS concerns
bcs_concerns = raw_bcs[raw_bcs["2026-02"] < 3.0]
if len(bcs_concerns) > 0:
    print("\nBody condition below 3.0 (most recent):")
    for _, row in bcs_concerns.iterrows():
        print(f"  {row['name']} ({row['animal_id']}): BCS {row['2026-02']}")

print("\n" + "=" * 50)
print(f"Total vet costs this period: ${total_vet_cost:.2f}")
print(f"Total treatment costs: ${total_treatment_cost:.2f}")
print(f"Combined health costs: ${total_vet_cost + total_treatment_cost:.2f}")

## Conclusion

Consistent health records transform flock management from guesswork into informed decision-making. The records in this notebook -- vaccinations, treatments, weights, body condition, and FAMACHA scores -- are the core data every Katahdin producer should be tracking.

### Key Takeaways

- Record every health event as it happens, not from memory later
- FAMACHA scores over time reveal which animals carry true parasite resistance
- Weight and ADG data identifies your best genetics for breeding stock
- Body condition scoring catches nutritional problems before they become emergencies
- Seasonal patterns help you plan preventive care instead of reacting to problems
- Organized records make vet visits more productive and cost-effective

### Next Steps

- [ ] Replace the sample flock data with your actual animals
- [ ] Enter your existing vaccination and treatment records
- [ ] Start FAMACHA scoring every 2-4 weeks during grazing season
- [ ] Weigh lambs at birth, 30, 60, and 90 days
- [ ] Body condition score ewes at breeding, mid-gestation, lambing, and weaning
- [ ] Review the seasonal pattern chart to build a preventive health calendar
- [ ] Print the summary report before your next vet visit
- [ ] Use FAMACHA and treatment data to inform breeding and culling decisions -- see [NSIP Sheep Genetics](nsip-sheep-genetics.ipynb) for integrating health records with breeding decisions
- [ ] Explore [Pasture & Grazing Management](pasture-grazing-management.ipynb) to connect seasonal health patterns with grazing rotations
- [ ] Back up your records regularly (this notebook file is your database)